# Quick Start: SAR + AIS Data Download

This notebook walks through pulling Sentinel-1 SAR scenes and matching AIS reports for the same area and time window. Run the cells from top to bottom to:
1. Set your area of interest and target date.
2. Query Copernicus Open Access Hub via `phidown` for matching Sentinel-1 Level-1 products.
3. Select a product, extract its sensing period and footprint, and reuse it for AIS filtering.
4. Download AIS tracks that fall inside the footprint and time window.
5. Inspect the resulting table and visualize the vessels on an interactive map.

> Install dependencies beforehand: `pip install phidown[ais,viz]`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Avoid cachelib prune race when serving tiles (Python 3.12)
from localtileserver.web import blueprint as _lts_blueprint, application as _lts_application
_lts_blueprint.cache.config.update({"CACHE_THRESHOLD": 1_000_000})
try:
    _lts_backend = _lts_application.cache.cache
except Exception:
    _lts_backend = None
else:
    if hasattr(_lts_backend, "_threshold"):
        _lts_backend._threshold = 1_000_000
from localtileserver.manager import AppManager as _LTSAppManager
_LTSAppManager._APP = None


/Users/roberto.delprete/Library/CloudStorage/OneDrive-ESA/Desktop/Repos/phidown/.venv/lib/python3.12/site-packages/localtileserver/_version.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [3]:
# Bring in helper classes for Copernicus search and AIS extraction
from phidown import CopernicusDataSearcher
from phidown.ais import download_ais_data

# Data manipulation and mapping helpers
import pandas as pd
import folium
from shapely import wkt as shapely_wkt
import localtileserver
from pathlib import Path
from typing import Union

import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_gcps, array_bounds
from rasterio.warp import calculate_default_transform, reproject

In [4]:
from pathlib import Path
from typing import Union

import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_gcps, array_bounds
from rasterio.warp import calculate_default_transform, reproject

# ----------------------------------------------------------------------
# Helper function to warp Sentinel-1 measurement TIFFs to EPSG:4326
# ----------------------------------------------------------------------
def warp_to_epsg4326(
    src_tiff: Union[str, Path],
    dst_tiff: Union[str, Path],
    *,
    resampling: Resampling = Resampling.bilinear,
) -> Path:
    """
    Reproject a Sentinel-1 measurement TIFF to EPSG:4326, honoring its GCPs.

    Parameters
    ----------
    src_tiff : str or Path
        Path to the original measurement TIFF inside the SAFE package.
    dst_tiff : str or Path
        Destination path for the geocoded GeoTIFF.
    resampling : rasterio.enums.Resampling, optional
        Resampling kernel (defaults to bilinear).

    Returns
    -------
    Path
        Absolute path to the geocoded raster.
    """
    src_path = Path(src_tiff).expanduser().resolve()
    dst_path = Path(dst_tiff).expanduser().resolve()
    dst_path.parent.mkdir(parents=True, exist_ok=True)

    with rasterio.Env(GDAL_GCP_USE_OK="YES"):
        with rasterio.open(src_path) as src:
            gcps, gcp_crs = src.gcps

            if src.crs:
                src_crs = src.crs
                src_transform = src.transform
            elif gcps and gcp_crs:
                src_crs = gcp_crs
                src_transform = from_gcps(gcps)
            else:
                raise ValueError("Source raster has neither a CRS nor GCPs.")

            bounds = array_bounds(src.height, src.width, src_transform)
            dst_crs = "EPSG:4326"
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src_crs, dst_crs, src.width, src.height, *bounds
            )

            profile = src.profile.copy()
            for key in ("affine", "transform"):
                profile.pop(key, None)  # avoid stale metadata
            profile.update(
                driver="GTiff",
                height=dst_height,
                width=dst_width,
                transform=dst_transform,
                crs=dst_crs,
                count=src.count,
                dtype=src.dtypes[0],
                compress="deflate",
            )

            with rasterio.open(dst_path, "w", **profile) as dst:
                for band in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, band),
                        destination=rasterio.band(dst, band),
                        src_transform=src_transform,
                        src_crs=src_crs,
                        dst_transform=dst_transform,
                        dst_crs=dst_crs,
                        resampling=resampling,
                    )

    with rasterio.open(dst_path, "r+") as dst:  # build overviews for faster tile access
        dim = min(dst.width, dst.height)
        factors = []
        factor = 2
        while dim // factor >= 1:
            factors.append(factor)
            factor *= 2
        if factors:
            dst.build_overviews(factors, Resampling.average)
            dst.update_tags(ns="rio_overview", resampling="average")

    return dst_path


You can make a WKT here: https://wktmap.com/

In [5]:
# ----------------------------------------------------------------------
# QUERY COPERNICUS FOR SENTINEL-1 PRODUCTS
# ----------------------------------------------------------------------

# Define area of interest (Antwerp region, Belgium) in WKT format
aoi_wkt = """POLYGON((4.2100 51.3700,4.4800 51.3700,4.5100 51.2900,4.4650 51.1700,4.2500 51.1700,4.1900 51.2500,4.2100 51.3700))"""

# Configure the search time window (full day by default)
target_date = "2025-08-28"  # YYYY-MM-DD format
start_time = "00:00:00"
start = f"{target_date}T{start_time}"
end_time = "23:00:00"
end = f"{target_date}T{end_time}"

print(f"Searching for data on {target_date}, between {start_time} and {end_time}")
print("Area: Port of Antwerp, Belgium")

# Query Copernicus for Sentinel-1 Level-1 products over the AOI and date range
searcher = CopernicusDataSearcher()
searcher.query_by_filter(
    collection_name='SENTINEL-1',
    product_type=None,
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=aoi_wkt,
    start_date=start,
    end_date=end,
    top=1000,
    attributes={'processingLevel': 'LEVEL1'}
)
df = searcher.execute_query()
df


Searching for data on 2025-08-28, between 00:00:00 and 23:00:00
Area: Port of Antwerp, Belgium
WKT polygon normalized: Whitespace and formatting corrected


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes
0,application/octet-stream,2a3b99de-497e-4ea4-985b-99d40411eba2,S1A_IW_SLC__1SDV_20250828T055900_20250828T0559...,application/octet-stream,8225072450,2025-08-28T07:00:00.465000Z,2025-08-28T07:07:16.405849Z,2025-08-28T07:25:35.958397Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_SLC__1S/2025/08/28/S...,"[{'Value': '6e106ae77cd2e211f78a4c041e58c556',...","{'Start': '2025-08-28T05:59:00.661693Z', 'End'...","geography'SRID=4326;POLYGON ((4.9782 49.45335,...","{'type': 'Polygon', 'coordinates': [[[4.9782, ...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,application/octet-stream,19bc7184-1a2d-4ce9-ade8-03248925b8c0,S1A_IW_GRDH_1SDV_20250828T055836_20250828T0559...,application/octet-stream,1027594958,2025-08-28T04:54:29.916167Z,2025-08-28T06:58:15.455863Z,2025-08-28T07:13:37.998750Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S-COG/2025/08/...,"[{'Value': 'c165d508dc3ebe12f6e32598248ca6bc',...","{'Start': '2025-08-28T05:58:36.832971Z', 'End'...",geography'SRID=4326;POLYGON ((5.471106 51.0028...,"{'type': 'Polygon', 'coordinates': [[[5.471106...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,application/octet-stream,f3cfe20f-8ebd-42e0-b35b-69b9208d174c,S1A_IW_GRDH_1SDV_20250828T055836_20250828T0559...,application/octet-stream,1773984888,2025-08-28T06:49:46.050000Z,2025-08-28T06:52:58.013907Z,2025-08-28T07:01:33.904451Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2025/08/28/S...,"[{'Value': 'e7d4e0f119160204f34fa8fa709621ae',...","{'Start': '2025-08-28T05:58:36.832971Z', 'End'...",geography'SRID=4326;POLYGON ((5.471106 51.0028...,"{'type': 'Polygon', 'coordinates': [[[5.471106...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,application/octet-stream,4283f9a1-fcc7-4b02-aa1e-0866422ae891,S1A_IW_SLC__1SDV_20250828T055835_20250828T0559...,application/octet-stream,8223940311,2025-08-28T06:59:28.484000Z,2025-08-28T07:05:32.092342Z,2025-08-28T07:29:25.652590Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_SLC__1S/2025/08/28/S...,"[{'Value': '0305a9e8a07200c8addb1301910d7a60',...","{'Start': '2025-08-28T05:58:35.840852Z', 'End'...",geography'SRID=4326;POLYGON ((5.45114 50.93614...,"{'type': 'Polygon', 'coordinates': [[[5.45114,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."


## Retrieve matching AIS data

In [6]:
# ----------------------------------------------------------------------
# DOWNLOAD A SENTINEL-1 PRODUCT
# ----------------------------------------------------------------------

# Select the first product, download it and reuse its sensing window and footprint
prod_idx = 2
sel_prod = df.iloc[prod_idx]
# searcher.download_product(eo_product_name=sel_prod['Name'], output_dir='./data', config_file='../.s5cfg')

In [7]:
# ----------------------------------------------------------------------
# REUSE SENSING WINDOW AND FOOTPRINT TO DOWNLOAD AIS MESSAGES
# ----------------------------------------------------------------------
# Extract timing metadata and footprint WKT from the product

content_date = sel_prod['ContentDate']
start_time = content_date['Start'].split('T')[1]
end_time = content_date['End'].split('T')[1]
aoi_wkt = sel_prod['Footprint'].split(";")[1].replace("'", "")

# Download AIS data that overlaps the Sentinel-1 sensing window and footprint
try:
    # Request AIS tracks filtered by both time window and area of interest
    ais_data = download_ais_data(
        start_date=target_date,
        start_time=start_time,  # Handles Z suffix and microseconds correctly
        end_time=end_time,      # Matches Sentinel-1 sensing period precisely
        aoi_wkt='POLYGON ((5.471106 51.002899, 5.967843 52.495514, 2.110601 52.909225, 1.739314 51.414181, 5.471106 51.002899))'
    )

    if not ais_data.empty:
        print(f"Actual data timespan: {ais_data['timestamp'].iloc[0]} to {ais_data['timestamp'].iloc[-1]}")
    else:
        print("No AIS detections matched the selected filters.")

except Exception as e:
    print(f"AIS download failed: {e}")
    print("Make sure you have installed: pip install phidown[ais]")

# Inspect the filtered AIS dataset
ais_data.head()

Added 3506 rows from 2025-08-28_ais.parquet
Final result: 3506 rows total with 25 columns
Actual data timespan: 2025-08-28 05:58:37 to 2025-08-28 05:59:01


,name,lat,lon,source_date,timestamp,mmsi,MMSI,TSTAMP,LATITUDE,LONGITUDE,...,NAME,CALLSIGN,TYPE,A,B,C,D,DRAUGHT,DEST,ETA
0,VEERHAVEN 13,51.88663,4.43319,2025-08-28,2025-08-28 05:58:37,244317050,244317050,2025-08-28 05:58:37+00:00,51.88663,4.43319,...,VEERHAVEN 13,PD6466,31.0,15.0,1.0,3.0,3.0,1.0,ROTTERDAM,00-00 24:60
1,ARIFE,51.35609,3.18211,2025-08-28,2025-08-28 05:58:37,314777000,314777000,2025-08-28 05:58:37+00:00,51.35609,3.18211,...,ARIFE,8PQO6,71.0,89.0,12.0,11.0,6.0,5.2,BEZEE,08-27 15:00
2,FINT,51.88894,4.31713,2025-08-28,2025-08-28 05:58:37,211804140,211804140,2025-08-28 05:58:37+00:00,51.88894,4.31713,...,FINT,DF2870,80.0,100.0,10.0,5.0,6.0,0.3,EXXON MOBILEROTTERDA,08-27 20:43
3,MELODY,51.39577,3.57370,2025-08-28,2025-08-28 05:58:37,211876350,211876350,2025-08-28 05:58:37+00:00,51.39577,3.57370,...,MELODY,DF8661,37.0,12.0,3.0,2.0,2.0,0.0,None,00-00 00:00
4,JULIA,51.88249,4.30608,2025-08-28,2025-08-28 05:58:37,244070891,244070891,2025-08-28 05:58:37+00:00,51.88249,4.30608,...,JULIA,PB6485,99.0,100.0,10.0,4.0,8.0,0.0,ROTTERDAM CALANDKANA,08-28 07:26


## Visualize Sentinel-1 footprint and AIS vessels

In [11]:
# ----------------------------------------------------------------------
# VISUALIZE SENTINEL-1 MEASUREMENT AND AIS DETECTIONS ON A MAP
# ----------------------------------------------------------------------

from localtileserver import TileClient, get_leaflet_tile_layer
from ipyleaflet import (
    Map, basemaps, LayersControl, FullScreenControl,
    ScaleControl, CircleMarker, LayerGroup, Popup
)
from ipywidgets import Layout, HTML

safe_package = Path(f"./data/{sel_prod['Name']}")
print(f"Using SAFE package at: {safe_package}")
measurement_dir = safe_package / "measurement"

# Verify if the measurement directory exists
if not measurement_dir.exists():
    raise FileNotFoundError(f"Measurement directory does not exist: {measurement_dir}")

# Check available files in the measurement directory (both .tif and .tiff extensions)
available_files = list(measurement_dir.glob("*.tif*"))
if not available_files:
    raise FileNotFoundError(f"No measurement GeoTIFF files found in {measurement_dir}. Please ensure the SAFE package is correctly downloaded and contains the required files.")

print(f"Found {len(available_files)} measurement files:")
for file in available_files:
    print(f"  {file.name}")

# Extract available polarizations from file names (look for vh/vv in filename)
available_polarizations = set()
for file in available_files:
    filename_lower = file.name.lower()
    if '-vh-' in filename_lower:
        available_polarizations.add('vh')
    elif '-vv-' in filename_lower:
        available_polarizations.add('vv')

print(f"Available polarizations: {available_polarizations}")

# Set polarization to an available one if the desired one is not found
polarization = "vh" if "vh" in available_polarizations else list(available_polarizations)[0]
print(f"Using polarization: {polarization}")

# Find files matching the selected polarization (exclude already geocoded files)
candidates = [f for f in available_files if f'-{polarization}-' in f.name.lower() and 'geocoded' not in f.name.lower()]
if not candidates:
    raise FileNotFoundError(f"No {polarization.upper()} measurement GeoTIFF found in {measurement_dir}")

src_tif = sorted(candidates)[0]
dst_tif = src_tif.with_name(f"{src_tif.stem}-geocoded{src_tif.suffix}")

tif = warp_to_epsg4326(src_tif, dst_tif)

# --- Using tileserver to serve the GeoTIFF as map tiles ---
client = TileClient(str(tif))
t = get_leaflet_tile_layer(client, indexes=1, vmin=0, vmax=400)

# --- Map widget: size + basemap (CartoDB Positron) ---
m = Map(
    center=client.center(),
    zoom=8,
    basemap=basemaps.CartoDB.Positron,
    layout=Layout(width="100%", height="900px"),
    scroll_wheel_zoom=True
)

# --- Add layers & controls ---
m.add(t)
m.add(LayersControl(position="topright"))
m.add(FullScreenControl())
m.add(ScaleControl(position="bottomleft"))

# --- Add AIS markers ---
if "ais_data" in locals() and not ais_data.empty:
    vessel_markers = []
    for row in ais_data.itertuples(index=False):
        popup_html = HTML(
            value=(
                f"<strong>{(row.name or 'Unknown').strip() or 'Unknown vessel'}</strong>"
                f"<br>MMSI: {row.mmsi}<br>{row.timestamp}"
            )
        )

        marker = CircleMarker(
            location=(float(row.lat), float(row.lon)),
            radius=4,
            color="#fff134",
            fill_color="#e31a1c",
            fill_opacity=0.5,
            weight=1,
        )
        marker.popup = popup_html
        vessel_markers.append(marker)

    ais_layer = LayerGroup(layers=vessel_markers, name="AIS detections")
    m.add_layer(ais_layer)
else:
    print("No AIS detections to display.")

m


Using SAFE package at: data/S1A_IW_GRDH_1SDV_20250828T055836_20250828T055901_060732_078EF5_FC4D.SAFE
Found 3 measurement files:
  s1a-iw-grd-vh-20250828t055836-20250828t055901-060732-078ef5-002.tiff
  s1a-iw-grd-vh-20250828t055836-20250828t055901-060732-078ef5-002-geocoded.tiff
  s1a-iw-grd-vv-20250828t055836-20250828t055901-060732-078ef5-001.tiff
Available polarizations: {'vv', 'vh'}
Using polarization: vh


Map(center=[51.965149499999995, 3.8265610000000003], controls=(ZoomControl(options=['position', 'zoom_in_text'…